# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


###Import necessary libraries for the model

In [1]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.layers import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

tf.reset_default_graph()
tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
#Get the data from keras dataset...
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
#Print the count of records in train and test set...
print ("x_train row count: ", len(x_train))
print ("y_train row count: ", len(y_train))
print ("x_test row count: ", len(x_test))
print ("y_test row count: ", len(y_test))

x_train row count:  50000
y_train row count:  50000
x_test row count:  10000
y_test row count:  10000


In [0]:
#As the target dataset has array of array of values, lets convert it to array of values...
y_train = np.array([x[0] for x in y_train])
y_test = np.array([x[0] for x in y_test])

In [0]:
#Split the train data into two with one having classess from 0 to 4 and two having classess from 5 to 9...
x_train0to4 = x_train[y_train<=4]
y_train0to4 = y_train[y_train<=4]
x_train5to9 = x_train[y_train>4]
y_train5to9 = y_train[y_train>4]

In [7]:
#Now get the count of rows in train after split...
print ("x_train0to4 row count: ", len(x_train0to4))
print ("y_train0to4 row count: ", len(y_train0to4))
print ("x_train5to9 row count: ", len(x_train5to9))
print ("y_train5to9 row count: ", len(y_train5to9))

x_train0to4 row count:  25000
y_train0to4 row count:  25000
x_train5to9 row count:  25000
y_train5to9 row count:  25000


In [0]:
#Split the test data into two with one having classess from 0 to 4 and two having classess from 5 to 9...
x_test0to4 = x_test[y_test<=4]
y_test0to4 = y_test[y_test<=4]
x_test5to9 = x_test[y_test>4]
y_test5to9 = y_test[y_test>4]

In [9]:
#Now get the count of rows in test after split...
print ("x_test0to4 row count: ", len(x_test0to4))
print ("y_test0to4 row count: ", len(y_test0to4))
print ("x_test5to9 row count: ", len(x_test5to9))
print ("y_test5to9 row count: ", len(y_test5to9))

x_test0to4 row count:  5000
y_test0to4 row count:  5000
x_test5to9 row count:  5000
y_test5to9 row count:  5000


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train0to4 = keras.utils.to_categorical(y_train0to4, 5)
y_test0to4 = keras.utils.to_categorical(y_test0to4, 5)

#As the second set have values from 5 to 9... when we do one hot encoding, it will end up with 10 index.
#To avoid it subtract the values by 5, so we have values only from 0 to 4, so while doing one hot encoding the index will be 5.
#Converting the y_train and y_test to have values 0 to 4
y_train5to9 = y_train5to9 - 5
y_test5to9 = y_test5to9 - 5 

y_train5to9 = keras.utils.to_categorical(y_train5to9, 5)
y_test5to9 = keras.utils.to_categorical(y_test5to9, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
x_train0to4 = x_train0to4.reshape(len(x_train0to4),32,32,3)
x_train5to9 = x_train5to9.reshape(len(x_train5to9),32,32,3)
x_test0to4 = x_test0to4.reshape(len(x_test0to4),32,32,3)
x_test5to9 = x_test5to9.reshape(len(x_test5to9),32,32,3)

x_train0to4 = x_train0to4/255
x_test0to4 = x_test0to4/255
x_train5to9 = x_train5to9/255
x_test5to9 = x_test5to9/255

In [12]:
model = Sequential()
model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Conv2D(32, (3,3), input_shape = (32, 32, 3), kernel_initializer = 'he_normal')) 
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=5, padding='same', kernel_initializer = 'he_normal'))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=4, padding='same', kernel_initializer = 'he_normal'))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer = 'he_normal'))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', kernel_initializer = 'he_normal'))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Dropout(0.2))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128, kernel_initializer = 'he_normal'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(Dense(units = 64, kernel_initializer = 'he_normal'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(Dropout(0.3))
model.add(Dense(units = 32, kernel_initializer = 'he_normal'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('elu'))
model.add(Dropout(0.3))
model.add(Dense(units = 5, activation = 'softmax')) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
#optimizer = Adam(lr=0.0001)
#opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
history = model.fit(x_train0to4, y_train0to4,
                    epochs=20, 
                    validation_data=(x_test0to4, y_test0to4),
                    verbose = 1,
                    initial_epoch=0,batch_size=100)

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 12s 463us/step - loss: 1.3424 - acc: 0.4665 - val_loss: 1.0381 - val_acc: 0.5838
Epoch 2/20
25000/25000 [==============================] - 9s 363us/step - loss: 1.0166 - acc: 0.5900 - val_loss: 0.8637 - val_acc: 0.6588
Epoch 3/20
25000/25000 [==============================] - 9s 363us/step - loss: 0.8857 - acc: 0.6518 - val_loss: 0.7404 - val_acc: 0.7136
Epoch 4/20
25000/25000 [==============================] - 9s 365us/step - loss: 0.7958 - acc: 0.6969 - val_loss: 0.6739 - val_acc: 0.7462
Epoch 5/20
25000/25000 [==============================] - 9s 366us/step - loss: 0.7389 - acc: 0.7244 - val_loss: 0.6268 - val_acc: 0.7658
Epoch 6/20
25000/25000 [==============================] - 9s 365us/step - loss: 0.6739 - acc: 0.7538 - val_loss: 0.5852 - val_acc: 0.7848
Epoch 7/20
25000/25000 [==============================] - 9s 365us/step - 

In [0]:
predictions_train = model.predict(x_train0to4)
predictions_test = model.predict(x_test0to4)

In [16]:
from sklearn import metrics
print ("Train Accuracy: ", metrics.accuracy_score(np.argmax(y_train0to4, axis=1), np.argmax(predictions_train, axis=1)))

Train Accuracy:  0.93112


In [17]:
print ("Test Accuracy: ", metrics.accuracy_score(np.argmax(y_test0to4, axis=1), np.argmax(predictions_test, axis=1)))

Test Accuracy:  0.8412


In [18]:
from sklearn.metrics import classification_report

print ("Classification Report for train data")
print(classification_report(np.argmax(y_train0to4, axis=1), np.argmax(predictions_train, axis=1)))

Classification Report for train data
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      5000
           1       0.99      0.99      0.99      5000
           2       0.94      0.84      0.89      5000
           3       0.96      0.87      0.91      5000
           4       0.87      0.97      0.92      5000

   micro avg       0.93      0.93      0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



In [19]:
print ("Classification Report for test data")
print(classification_report(np.argmax(y_test0to4, axis=1), np.argmax(predictions_test, axis=1)))

Classification Report for test data
              precision    recall  f1-score   support

           0       0.82      0.92      0.87      1000
           1       0.96      0.96      0.96      1000
           2       0.82      0.70      0.75      1000
           3       0.86      0.71      0.78      1000
           4       0.76      0.91      0.83      1000

   micro avg       0.84      0.84      0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000



### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [20]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

batch_normalization_1
False
conv2d_1
False
batch_normalization_2
False
activation_1
False
max_pooling2d_1
False
dropout_1
False
conv2d_2
False
batch_normalization_3
False
activation_2
False
max_pooling2d_2
False
dropout_2
False
conv2d_3
False
batch_normalization_4
False
activation_3
False
max_pooling2d_3
False
dropout_3
False
conv2d_4
False
batch_normalization_5
False
activation_4
False
max_pooling2d_4
False
dropout_4
False
conv2d_5
False
batch_normalization_6
False
activation_5
False
max_pooling2d_5
False
dropout_5
False
flatten_1
False
dense_1
True
activation_6
False
dense_2
True
activation_7
False
dropout_6
False
dense_3
True
activation_8
False
dropout_7
False
dense_4
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
input_shape = model.output_shape[1]

model5to9 = Sequential()
model5to9.add(model)

model5to9.add(Dense(512, activation='elu', input_dim=input_shape))
model5to9.add(Dropout(0.3))
model5to9.add(Dense(248, activation='elu', input_dim=input_shape))
model5to9.add(Dropout(0.3))
model5to9.add(Dense(124, activation='elu', input_dim=input_shape))
model5to9.add(Dropout(0.3))
model5to9.add(Dense(68, activation='elu'))
model5to9.add(Dropout(0.3))
model5to9.add(Dense(32, activation='elu'))
model5to9.add(Dropout(0.3))
model5to9.add(Dense(units = 5, activation = 'softmax')) 

In [26]:
model5to9.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
history = model5to9.fit(x_train5to9, y_train5to9,
                    epochs=20, 
                    validation_data=(x_test5to9, y_test5to9),
                    verbose = 1,
                    initial_epoch=0,batch_size=100)

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 8s 313us/step - loss: 0.7776 - acc: 0.7306 - val_loss: 0.5937 - val_acc: 0.7820
Epoch 2/20
25000/25000 [==============================] - 5s 206us/step - loss: 0.7346 - acc: 0.7404 - val_loss: 0.5441 - val_acc: 0.7966
Epoch 3/20
25000/25000 [==============================] - 5s 200us/step - loss: 0.7407 - acc: 0.7344 - val_loss: 0.5532 - val_acc: 0.7984
Epoch 4/20
25000/25000 [==============================] - 5s 199us/step - loss: 0.7259 - acc: 0.7426 - val_loss: 0.5820 - val_acc: 0.7874
Epoch 5/20
25000/25000 [==============================] - 5s 199us/step - loss: 0.7280 - acc: 0.7403 - val_loss: 0.5698 - val_acc: 0.7912
Epoch 6/20
25000/25000 [==============================] - 5s 199us/step - loss: 0.7261 - acc: 0.7416 - val_loss: 0.5691 - val_acc: 0.7896
Epoch 7/20
25000/25000 [==============================] - 5s 199us/step - loss: 0.7224 - acc: 0.7428 - val_loss: 0.5655 - v

Train Bottleneck Features: (25000, 5) 	Validation Bottleneck Features: (5000, 5)


In [0]:
predictions_test = model.predict(x_test5to9)

In [24]:
print ("Test Accuracy: ", metrics.accuracy_score(np.argmax(y_test5to9, axis=1), np.argmax(predictions_test, axis=1)))

Test Accuracy:  0.7668


In [0]:
print ("Classification Report for test data")
print(classification_report(np.argmax(y_test0to4, axis=1), np.argmax(predictions_test, axis=1)))

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

In [0]:
twenty_train.target_names

In [0]:
twenty_train.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score